## Deliverable 2. Create a Customer Travel Destinations Map.

In [36]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [37]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,77.16,94,20,8.05,few clouds
1,1,Georgetown,MY,5.4112,100.3354,85.95,74,20,2.30,few clouds
2,2,Airai,TL,-8.9266,125.4092,61.20,92,100,2.44,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,46.06,57,75,8.05,broken clouds
4,4,Castro,BR,-24.7911,-50.0119,71.73,79,77,10.54,light rain


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 72
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Vaini,TO,-21.2000,-175.2000,77.16,94,20,8.05,few clouds
1,1,Georgetown,MY,5.4112,100.3354,85.95,74,20,2.30,few clouds
20,20,Atuona,PF,-9.8000,-139.0333,78.33,79,28,16.80,light rain
24,24,Lorengau,PG,-2.0226,147.2712,81.79,74,10,6.42,clear sky
25,25,Mogadishu,SO,2.0371,45.3438,85.05,67,5,14.83,clear sky
26,26,Oistins,BB,13.0667,-59.5333,84.29,58,20,19.57,few clouds
27,27,Puerto Ayora,EC,-0.7393,-90.3518,78.55,94,56,1.99,broken clouds
28,28,Yining,CN,29.0361,114.5636,75.25,68,100,3.02,overcast clouds
29,29,Sibolga,ID,1.7427,98.7792,78.37,87,57,2.33,broken clouds
31,31,La Primavera,CO,5.4906,-70.4092,82.63,66,99,4.88,overcast clouds


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                272
City                   272
Country                268
Lat                    272
Lng                    272
Max Temp               272
Humidity               272
Cloudiness             272
Wind Speed             272
Current Description    272
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [42]:
clean_df.count()

City_ID                268
City                   268
Country                268
Lat                    268
Lng                    268
Max Temp               268
Humidity               268
Cloudiness             268
Wind Speed             268
Current Description    268
dtype: int64

In [43]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
1,Georgetown,MY,85.95,few clouds,5.4112,100.3354,
20,Atuona,PF,78.33,light rain,-9.8000,-139.0333,
24,Lorengau,PG,81.79,clear sky,-2.0226,147.2712,
25,Mogadishu,SO,85.05,clear sky,2.0371,45.3438,
26,Oistins,BB,84.29,few clouds,13.0667,-59.5333,
27,Puerto Ayora,EC,78.55,broken clouds,-0.7393,-90.3518,
28,Yining,CN,75.25,overcast clouds,29.0361,114.5636,
29,Sibolga,ID,78.37,broken clouds,1.7427,98.7792,
31,La Primavera,CO,82.63,overcast clouds,5.4906,-70.4092,


In [44]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
 
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan
        

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [45]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
1,Georgetown,MY,85.95,few clouds,5.4112,100.3354,Cititel Penang
20,Atuona,PF,78.33,light rain,-9.8000,-139.0333,Villa Enata
24,Lorengau,PG,81.79,clear sky,-2.0226,147.2712,Lorengau Harbourside Hotel
25,Mogadishu,SO,85.05,clear sky,2.0371,45.3438,Hotel Juba
26,Oistins,BB,84.29,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
27,Puerto Ayora,EC,78.55,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
28,Yining,CN,75.25,overcast clouds,29.0361,114.5636,Xiushui Hotel
29,Sibolga,ID,78.37,broken clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
31,La Primavera,CO,82.63,overcast clouds,5.4906,-70.4092,La Riviera Hotel Colonial


In [46]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [47]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))